In [12]:
import numpy as np
import pandas as pd
import json
import requests
import urllib
import httplib
import base64

In [30]:
sample_text = {0 : "Such a lovely hackathon we are in!", 1 : "We are definitely going to", 2 : "win and to go Paris!"}
pd_text = pd.DataFrame(pd.Series(sample_text).rename("text"))
pd_text.head()

,text
0,Such a lovely hackathon we are in!
1,We are definitely going to
2,win and to go Paris!


In [3]:
api_endpoint = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0"
api_key_one = "28d6776ec31844a1aeb1095be8d99192"
api_key_two = "ec713e3e3ed64186aa185d02acbf5014"

In [18]:
def generate_json_request(pd_text):
    request = {}
    request["documents"] = []
    
    for index, row in pd_text.iterrows():
        phrase = {}
        phrase["lang"] = "en"
        phrase["id"] = index
        phrase["text"] = row["text"]
        request["documents"].append(phrase)
    return json.dumps(request)

json_request = generate_json_request(pd_text)
print json_request

{"documents": [{"lang": "en", "text": "Such a lovely hackathon we are in!", "id": 0}, {"lang": "en", "text": "We definitely going to", "id": 1}, {"lang": "en", "text": "win and to go Paris!", "id": 2}]}


In [21]:
def process_request(json_request):
    header = {
    'Accept' : 'application/json',
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '28d6776ec31844a1aeb1095be8d99192',
    }
    try:
        conn = httplib.HTTPSConnection('westus.api.cognitive.microsoft.com')
        
        conn.request("POST", "/text/analytics/v2.0/sentiment", json_request, header)
        response = conn.getresponse()
        sentiment_data = response.read()
        
        conn.request("POST", "/text/analytics/v2.0/keyPhrases", json_request, header)
        response = conn.getresponse()
        key_phrase_data = response.read()
        
        conn.close()
        return sentiment_data, key_phrase_data
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        return None    

In [22]:
sentiment_data, key_phrase_data = process_request(json_request)

In [32]:
def add_text_analysis(base_df, sentiment_data, key_phrase_data):
    sentiment_dict = {}
    for entry in json.loads(sentiment_data)["documents"]:
        sentiment_dict[int(entry["id"])] = entry["score"]

    key_phrase_dict = {}
    for entry in json.loads(key_phrase_data)["documents"]:
        key_phrase_dict[int(entry["id"])] = entry["keyPhrases"]

    final_pd = pd.DataFrame(base_df)
    final_pd["sentiment_score"] = pd.Series(sentiment_dict)
    final_pd["key_phrases"] = pd.Series(key_phrase_dict)
    return final_pd

In [33]:
def main(base_df):
    json_request = generate_json_request(base_df)
    sentiment_data, key_phrase_data = process_request(json_request)
    final_df = add_text_analysis(base_df, sentiment_data, key_phrase_data)
    return final_pd

In [34]:
final_pd = main(pd_text)

In [35]:
final_pd

,text,sentiment_score,key_phrases
0,Such a lovely hackathon we are in!,0.967976,[lovely hackathon]
1,We are definitely going to,0.950928,[]
2,win and to go Paris!,0.870765,[win]
